In [ ]:
import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DataEBPF.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               82332 non-null  int64  
 1   dur              82332 non-null  float64
 2   proto            82332 non-null  object 
 3   service          82332 non-null  object 
 4   spkts            82332 non-null  int64  
 5   dpkts            82332 non-null  int64  
 6   sbytes           82332 non-null  int64  
 7   dbytes           82332 non-null  int64  
 8   sinpkt           82332 non-null  float64
 9   dinpkt           82332 non-null  float64
 10  tcprtt           82332 non-null  float64
 11  synack           82332 non-null  float64
 12  ackdat           82332 non-null  float64
 13  smean            82332 non-null  int64  
 14  dmean            82332 non-null  int64  
 15  is_sm_ips_ports  82332 non-null  int64  
 16  attack_cat       82332 non-null  object 
 17  label       

In [ ]:
df.attack_cat.value_counts()
df.label.value_counts()

label
1    45332
0    37000
Name: count, dtype: int64

In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import TensorDataset

#numerical_columns = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'sinpkt', 'dinpkt', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'is_sm_ips_ports', 'label']  # List your numerical column names here
#label_columns = ['proto', 'service', 'attack_cat']

#numerical columns (scaling)
#scaler = StandardScaler()
#df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

#label columns (encoding)
#label_encoders = {}
#for col in label_columns:
#    label_encoders[col] = LabelEncoder()
#    df[col] = label_encoders[col].fit_transform(df[col])


#numerical_data = torch.tensor(df[numerical_columns].values, dtype=torch.float32)

#label_tensors = [torch.tensor(df[col].values, dtype=torch.long) for col in label_columns]

#labels_combined = torch.cat(label_tensors, dim=0)
#labels_combined = labels_combined.view(-1, 3)

#dataset = TensorDataset(numerical_data, labels_combined)
df['label']


0        0
1        0
2        0
3        0
4        0
        ..
82327    0
82328    0
82329    0
82330    0
82331    0
Name: label, Length: 82332, dtype: int64

In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import TensorDataset

numerical_columns = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'sinpkt', 'dinpkt', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'is_sm_ips_ports']  # List your numerical column names here
label_columns = ['proto', 'service', 'attack_cat']

#numerical columns (scaling)
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

#label columns (encoding)
label_encoders = {}
for col in label_columns:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

x=['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'sinpkt', 'dinpkt', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'is_sm_ips_ports', 'attack_cat','proto', 'service']
y=['label']
print(df['label'])
#X_value=df[x]
#Y_value=df[y]

#print(df[numerical_columns])
#print(df[label_columns])
#print(df[numerical_columns].dtypes)

#X_data = torch.tensor(X_value.values, dtype=torch.float32)

#Y_data= torch.tensor(Y_value.values, dtype=torch.long)
#Y_data=Y_data.view(-1,1)

#X_data.shape
#Y_data.shape
#Y_data.shape
#labels_combined = torch.cat(label_tensors, dim=0)
#labels_combined = labels_combined.view(-1, 3)

0        0
1        0
2        0
3        0
4        0
        ..
82327    0
82328    0
82329    0
82330    0
82331    0
Name: label, Length: 82332, dtype: int64


In [ ]:

numerical_data = torch.tensor(df[x].values, dtype=torch.float32)
label_tensors = torch.tensor(df[y].values, dtype=torch.long)

#labels_combined = torch.cat(label_tensors, dim=0)
#labels_combined = labels_combined.view(-1, 3)

datasett = TensorDataset(numerical_data, label_tensors)

In [ ]:
print(label_tensors)

tensor([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]])


In [ ]:
print(numerical_data.shape)
print(label_tensors.shape)


torch.Size([82332, 16])
torch.Size([82332, 1])


In [ ]:
import torch.nn.functional as F

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        print(out.shape)
        out = self.fc(out[:, -1, :])
        return out

# Assuming you already have your data loaded into numerical_data and label_tensors

# Creating a dataset
dataset = TensorDataset(numerical_data, label_tensors)

# Define batch size and split sizes
batch_size = 64
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Splitting the dataset into train and validation sets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Creating data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Model, criterion, and optimizer initialization
model = LSTMClassifier(input_size=16, hidden_size=128, num_layers=2, num_classes=1)  # Adjust num_classes accordingly
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

In [ ]:

for inputs, labels in train_loader:
    optimizer.zero_grad()
    inputs = inputs.unsqueeze(1)
    outputs = model(inputs)


    outputs = F.softmax(outputs, dim=1)

    print("Outputs:", outputs)
    print("Labels:", labels)
    #  Calculating accuracy of the model

    _, predicted = torch.max(outputs, 1)  # Get the index of the maximum value
    correct = (predicted == labels).sum().item()  # Count correct predictions
    total = labels.size(0)  # Get the total number of labels
    accuracy = correct / total


    print("Outputs:", outputs)
    print("Labels:", labels)
    print("Accuracy:", accuracy)

    labels = labels.float()  # Convert labels to float for loss calculation (if needed)

    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()



    #labels = labels.float()



    #loss = criterion(outputs, labels)

    #loss.backward()
    #optimizer.step()



Streaming output truncated to the last 5000 lines.
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0]])
Accuracy: 23.0
torch.Size([64, 1, 128])
Outputs: tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
      

## BURRRR

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import TensorDataset, DataLoader, random_split

# Define the Focal Loss class
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.alpha is not None:
            focal_loss = self.alpha * focal_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/DataEBPF.csv')

# Define numerical and label columns
numerical_columns = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'sinpkt', 'dinpkt', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'is_sm_ips_ports']
label_columns = ['proto', 'service', 'attack_cat']

# Scaling numerical columns
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Encoding label columns
label_encoders = {}
for col in label_columns:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Define feature columns and target column
x = numerical_columns + label_columns
y = ['label']

# Convert data to PyTorch tensors
numerical_data = torch.tensor(df[x].values, dtype=torch.float32)
label_tensors = torch.tensor(df[y].values, dtype=torch.long)

# Create dataset
dataset = TensorDataset(numerical_data, label_tensors)

# Define batch size and split sizes
batch_size = 64
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Split dataset into train and validation sets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the LSTMClassifier class
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Model, criterion, and optimizer initialization
model = LSTMClassifier(input_size=len(x), hidden_size=128, num_layers=2, num_classes=2)  # Assuming 2 classes for binary classification
criterion = FocalLoss(gamma=2, alpha=None, reduction='mean')  # Focal Loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)


Mounted at /content/drive
Loss: 0.17493335902690887
Accuracy: 0.390625
Loss: 0.17445138096809387
Accuracy: 0.390625
Loss: 0.17429059743881226
Accuracy: 0.390625
Loss: 0.17348015308380127
Accuracy: 0.453125
Loss: 0.1727587878704071
Accuracy: 0.578125
Loss: 0.17339526116847992
Accuracy: 0.453125
Loss: 0.17315234243869781
Accuracy: 0.53125
Loss: 0.1732342392206192
Accuracy: 0.484375
Loss: 0.17330674827098846
Accuracy: 0.453125
Loss: 0.17323461174964905
Accuracy: 0.5
Loss: 0.17316952347755432
Accuracy: 0.515625
Loss: 0.17296338081359863
Accuracy: 0.578125
Loss: 0.17302100360393524
Accuracy: 0.53125
Loss: 0.17292775213718414
Accuracy: 0.546875
Loss: 0.17347002029418945
Accuracy: 0.453125
Loss: 0.17274126410484314
Accuracy: 0.5625
Loss: 0.17268288135528564
Accuracy: 0.5625
Loss: 0.17214857041835785
Accuracy: 0.625
Loss: 0.1712857335805893
Accuracy: 0.6875
Loss: 0.17304305732250214
Accuracy: 0.515625
Loss: 0.17301534116268158
Accuracy: 0.515625
Loss: 0.1721053570508957
Accuracy: 0.578125
Loss

In [ ]:
# Inside the training loop
for inputs, labels in train_loader:
    optimizer.zero_grad()
    inputs = inputs.unsqueeze(1)
    outputs = model(inputs)
    outputs = F.softmax(outputs, dim=1)
    labels = labels.squeeze(1)


    # Calculating loss
    loss = criterion(outputs, labels)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Calculate accuracy
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total

    # Convert accuracy to percentage
    accuracy_percent = accuracy * 100

    print("Loss:", loss.item())
    print("Accuracy:", accuracy_percent,"%")


Loss: 0.0561550036072731
Accuracy: 95.3125 %
Loss: 0.04472127556800842
Accuracy: 96.875 %
Loss: 0.046054087579250336
Accuracy: 96.875 %
Loss: 0.05587613582611084
Accuracy: 95.3125 %
Loss: 0.034338779747486115
Accuracy: 98.4375 %
Loss: 0.055845778435468674
Accuracy: 95.3125 %
Loss: 0.04808779060840607
Accuracy: 95.3125 %
Loss: 0.04501601681113243
Accuracy: 96.875 %
Loss: 0.08283865451812744
Accuracy: 90.625 %
Loss: 0.038430068641901016
Accuracy: 98.4375 %
Loss: 0.06712861359119415
Accuracy: 93.75 %
Loss: 0.06665927171707153
Accuracy: 93.75 %
Loss: 0.07777377218008041
Accuracy: 92.1875 %
Loss: 0.036482080817222595
Accuracy: 98.4375 %
Loss: 0.047193851321935654
Accuracy: 96.875 %
Loss: 0.05609679967164993
Accuracy: 95.3125 %
Loss: 0.06634669750928879
Accuracy: 93.75 %
Loss: 0.06670989096164703
Accuracy: 93.75 %
Loss: 0.024073541164398193
Accuracy: 100.0 %
Loss: 0.056858524680137634
Accuracy: 95.3125 %
Loss: 0.034449636936187744
Accuracy: 98.4375 %
Loss: 0.05547112971544266
Accuracy: 95.31

In [ ]:
print('hi')

hi
